In [ ]:
# Declare calling parameter types & definitons

import numpy as np
dbl = np.float64

atomic_mass_C = 12.011  # Carbon mass

i = 0
j = 0
k = 0
l = 0
istat = 0
natms = 0
natms_1 = 0
natms_2 = 0
natms_3 = 0

a_x, a_y, a_z = 0.0, 0.0, 0.0
b_x, b_y, b_z = 0.0, 0.0, 0.0
c_x, c_y, c_z = 0.0, 0.0, 0.0
d_ij = 0.0
sf = 0.0  # Scaling factor (sf) in CONTCAR
total_mass = 0.0
com_x, com_y, com_z = 0.0, 0.0, 0.0  # Coordinates of the centre of mass

import numpy as np

# Initialize arrays
xxx = np.empty(0, dtype=dbl)
yyy = np.empty(0, dtype=dbl)
zzz = np.empty(0, dtype=dbl)
vxx = np.empty(0, dtype=dbl)
vyy = np.empty(0, dtype=dbl)
vzz = np.empty(0, dtype=dbl)
rx = np.empty((0, 0), dtype=dbl)
ry = np.empty((0, 0), dtype=dbl)
rz = np.empty((0, 0), dtype=dbl)
vx = np.empty((0, 0), dtype=dbl)
vy = np.empty((0, 0), dtype=dbl)
vz = np.empty((0, 0), dtype=dbl)

r_i = np.zeros(3, dtype=float)
r_j = np.zeros(3, dtype=float)
r_ij = np.zeros(3, dtype=float)

h = np.zeros((3, 3), dtype=float)

# Function to read the CONTCAR file
def read_contcar(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return lines

# Function to process the CONTCAR file
def process_contcar(lines):
    global sf, a_x, a_y, a_z, b_x, b_y, b_z, c_x, c_y, c_z, atm_1, atm_2, atm_3, natms_1, natms_2, natms_3, total_atoms, coordinates

    sf = float(lines[1].strip())
    a_x, a_y, a_z = map(float, lines[2].strip().split())
    b_x, b_y, b_z = map(float, lines[3].strip().split())
    c_x, c_y, c_z = map(float, lines[4].strip().split())
    
    atm_1, atm_2, atm_3 = lines[5].strip().split()
    natms_1, natms_2, natms_3 = map(int, lines[6].strip().split())
    
    total_atoms = natms_1 + natms_2 + natms_3
    
    coordinates = []
    string = []
    for line in lines[9:9+total_atoms]:
        parts = line.strip().split()
        coord = list(map(float,parts[:3]))
        strings = parts[3:]
        coordinates.append(coord)
        string.append(strings)

    coordinates_atm1=coordinates[:natms_1]
    coordinates_atm2=coordinates[natms_1:natms_1+natms_2]
    coordinates_atm3=coordinates[natms_1+natms_2:]
    
    return np.array(coordinates_atm3), total_atoms
    
# Function to compute the center of mass
def compute_center_of_mass(coords):
    total_mass = atomic_mass_C * natms_3
    com_x = np.sum((coords[:, 0])*atomic_mass_C) / total_mass
    com_y = np.sum((coords[:, 1])*atomic_mass_C) / total_mass
    com_z = np.sum((coords[:, 2])*atomic_mass_C) / total_mass
    return com_x, com_y, com_z

# Main program
if __name__ == "__main__":
    try:
        contcar_lines = read_contcar('CONTCAR')
        coordinates_atm3, total_atoms = process_contcar(contcar_lines)

        # Allocate memory for arrays based on number of atoms
        xxx = np.zeros(total_atoms, dtype=dbl)
        yyy = np.zeros(total_atoms, dtype=dbl)
        zzz = np.zeros(total_atoms, dtype=dbl)
        vxx = np.zeros(total_atoms, dtype=dbl)
        vyy = np.zeros(total_atoms, dtype=dbl)
        vzz = np.zeros(total_atoms, dtype=dbl)
        rx = np.zeros((3, total_atoms), dtype=dbl)
        ry = np.zeros((3, total_atoms), dtype=dbl)
        rz = np.zeros((3, total_atoms), dtype=dbl)
        vx = np.zeros((3, total_atoms), dtype=dbl)
        vy = np.zeros((3, total_atoms), dtype=dbl)
        vz = np.zeros((3, total_atoms), dtype=dbl)
        h = np.zeros((3, 3), dtype=dbl)

        # Initialize matrix h containing crystallographic vectors a, b and c

        h = [
            [a_x, a_y, a_z],
            [b_x, b_y, b_z],
            [c_x, c_y, c_z]
        ]

        # Store coordinates into respective arrays
        for i in range(len(coordinates_atm3)):
            rx[0, i], ry[0, i], rz[0, i] = coordinates_atm3[i]
        
        # Compute the center of mass
        com_x, com_y, com_z = compute_center_of_mass(coordinates_atm3)
        print(f"Center of Mass: ({com_x}, {com_y}, {com_z})")

        # Deallocate arrays
        del xxx, yyy, zzz, vxx, vyy, vzz, rx, ry, rz, vx, vy, vz

    except FileNotFoundError:
        print("Error: 'CONTCAR' file not found.")
    except Exception as e:
        print(f"Error reading 'CONTCAR' file: {e}")




# Shoot new carbon atom
import random

def read_new_contcar(filename):
    with open(filename, 'r') as file:
        new_lines = file.readlines()
    return new_lines

def process_new_contcar(new_lines):
    global atm_1, atm_2, atm_3, natms_1, natms_2, natms_3, total_atoms, coordinates, exisiting_z_atoms
    
    atm_1, atm_2, atm_3 = new_lines[5].strip().split()
    natms_1, natms_2, natms_3 = map(int, new_lines[6].strip().split())
    
    total_atoms = natms_1 + natms_2 + natms_3
    
    coordinates = []
    exisiting_z_atoms = []
    string = []
    for line in new_lines[9:9+total_atoms]:
        parts = line.strip().split()
        coord = list(map(float,parts[:3]))
        z_coord = list(map(float,parts[2:3]))
        strings = parts[3:]
        coordinates.append(coord)
        exisiting_z_atoms.append(z_coord)
        string.append(strings)
    
    return np.array(coordinates), total_atoms, np.array(exisiting_z_atoms)
    
def add_atom(existing_atoms, min_distance, max_distance_general, max_distance_specific, filter_min=0.2, filter_max=0.8, attempts=2000000):
    while True:
        for _ in range(attempts):
            max_attempts = 2000000
            for attempt in range(max_attempts):
                shoot_x = random.uniform(0, 1)
                shoot_y = random.uniform(0, 1)
                shoot_z = random.uniform(0.25, 0.9)
                if shoot_x > (com_x/2) and shoot_x < com_x+(com_x/2) and shoot_y > (com_y/2) and shoot_y < com_y+(com_y/2) and shoot_z < com_z:
                    continue
                new_atom= np.array([shoot_x,shoot_y,shoot_z])
    
            filtered_atoms = existing_atoms[
                np.logical_and.reduce((
                    existing_atoms[:,0],
                    existing_atoms[:,1],
                    existing_atoms[:,2] >= filter_min, existing_atoms[:,2] <= filter_max
                ))
            ]


            scaled_new_atom = np.array([new_atom[0]*a_x, new_atom[1]*b_y, new_atom[2]*c_z])
            scaled_existing_atoms = filtered_atoms*np.array([a_x,b_y,c_z])
            distances = np.linalg.norm(scaled_existing_atoms - scaled_new_atom, axis=1)

            specific_range = (distances >= min_distance) & (distances <= max_distance_specific)
            if np.any(specific_range):
    
                return new_atom, True
            else:
                return ValueError(f'no suitable positions after maximum attempts.'), False

new_contcar_lines = read_new_contcar('CONTCAR')        
existing_atoms, total_atoms, exisiting_z_atoms = process_new_contcar(new_contcar_lines)

min_distance = 1.6
max_distance_general = 25
max_distance_specific = 1.9

new_atom, output = add_atom(existing_atoms, min_distance, max_distance_general, max_distance_specific, filter_min=0.2, filter_max=0.8)
print(f'New atom postition: {new_atom}')

if output==True:
    with open('CONTCAR', 'r') as file:
        lines = file.readlines()
        new_line = f'  {new_atom[0]:.16f}  {new_atom[1]:.16f}  {new_atom[2]:.16f}\n'
        lines.insert(total_atoms+9, new_line)
        update_count = natms_3 + 1
        lines[6] = f'    {natms_1}    {natms_2}    {update_count}\n'
#        velocity ='  0.00000000E+00  0.00000000E+00  0.00000000E+00\n'
#        lines.append(velocity)
    with open('POSCAR', 'w') as file:
        file.writelines(lines)